# Implementación 
### Equipo: 

In [1]:
# Parámetros generales del notebook
ruta_bases = 'bases/'
sample_size = 200000
cv = 3

In [2]:
# Preparación ambiente AWS SageMaker
#!pip install --upgrade pip
#!pip install xgboost
#!pip install lightgbm
#!pip install -U scikit_learn
#!pip install -U pandas

In [5]:
# Importación de librerías
import json
import glob
import pandas as pd
import numpy as np
#import missingno as msgno
import matplotlib.pyplot as plt
import funciones as fn
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from ml_classes import PrepML, MLModel
from matplotlib.pyplot import rcParams
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge,SGDRegressor
from lib.get_nhtsa_json import get_nhtsa_json

/Users/fernando/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/fernando/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/fernando/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [6]:
# Parámetros generales para plots
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 15, 8
# Semilla pseudo-aleatoria
rd_seed = 1234

# 0. Obtener la información

Esta es una demostración del proceso realizado por `get_features.py` para poder obtener información relacionada con el `Vin` de los vehículos a través de una API. El proceso consite en los siguientes pasos:
* 1.- Extraer todos los `Vin` de la base completa `true_car_listings.csv`.
* 2.- Requerir a través de la Api 'chunks' de 50 registros por cada petición.
* 3.- Guardar en la memoria el json en formato texto, agregando 50 registros por cada iteración.
* 4.- Una vez completada las iteraciones guardar en formato json todos los registros requeridos.

In [ ]:
# Requerimos todos los 'Vin'    
all_vins = pd.read_csv(f'{ruta_bases}true_car_listings.csv')['Vin'].to_list()
# Parámetros de muestra
start = 8
end = 8

json_text = '['
for i in range(start, end+1):
    # Generar requerimiento con 50 registros Vin
    vin_list = all_vins[50 * (i - 1):50 * i]
    json_text += get_nhtsa_json(vin_list, i)

# Cerrar lista de Json
json_text = json_text[:-2] + ']'
# Exportar resultados a archivo json
with open(f'api_test/data_{start}_{end}.json', 'w') as json_file:
    json_file.write(json_text)

Paralelamente, en base a una muestra de la base total, se definió el primer filtro de variables requeridas a través la API: que tengan menos del 10% de datos perdidos, las cuales se presentan a continaución:

In [ ]:
cols = ['AirBagLocFront', 'BodyClass', 'BusFloorConfigType', 'BusType',
       'CustomMotorcycleType', 'DisplacementCC', 'DisplacementCI',
       'DisplacementL', 'Doors', 'EngineCylinders', 'EngineHP', 'EngineKW',
       'ErrorCode', 'ErrorText', 'FuelTypePrimary', 'Make', 'Manufacturer',
       'ManufacturerId', 'Model', 'ModelYear', 'MotorcycleChassisType',
       'MotorcycleSuspensionType', 'PlantCity', 'PlantCountry', 'TPMS',
       'TrailerBodyType', 'TrailerType', 'VIN', 'VehicleType']

Con estas columnas seleccionadas, se procede a importar los archivos json (varios en el proceso original) para luego mapearlos para retraer solo aquellas columnas, creando un DataFrame con ellas y luego exportarlas en un csv.

In [ ]:
# Importación de archivo json
filenames = glob.glob('api_test/*.json')
json_list = []

for filename in filenames:
    print(filename)
    with open(filename, 'r') as file:
        # Mapeamos considerando solo las columnas seleccionadas
        data = list(map(fn.get_info, 
                        json.loads(file.read())
                       )
                   )
    json_list += data

# Creación y exortación de DataFrame con features extraídos
data_json = pd.DataFrame(data=json_list,
                         columns=cols)
data_json.to_csv('api_test/data_api.csv')

# 1. Creación del Dataset 

## 1.1 Bases Originales

In [7]:
# Importación de las bases para muestras de entrenamiento y prueba
df_train = pd.read_csv(f'{ruta_bases}true_cars_train.csv',
                       delimiter=";")
df_test = pd.read_csv(f'{ruta_bases}true_cars_test.csv',
                      delimiter=";")
# Dimensiones de las bases
print(f'Base Train: {df_train.shape}\nBase Test: {df_test.shape}')

Base Train: (639145, 8)
Base Test: (212977, 8)


In [8]:
# Información general de muestra de entrenamiento
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639145 entries, 0 to 639144
Data columns (total 8 columns):
Price      639145 non-null int64
Year       639145 non-null int64
Mileage    639145 non-null int64
City       639145 non-null object
State      639145 non-null object
Vin        639145 non-null object
Make       639145 non-null object
Model      639145 non-null object
dtypes: int64(3), object(5)
memory usage: 39.0+ MB


In [9]:
# Creación de atributo 'sample'
df_train['sample'] = 'train'
df_test['sample'] = 'test'

In [10]:
# Unión de ambas bases
df_data = pd.concat([df_train, df_test])
print(f'Base Data: {df_data.shape}')

Base Data: (852122, 9)


## 1.2 Base API

In [11]:
# Importación de la base extraída por el requerimiento a la api
df_api = pd.read_csv(f'{ruta_bases}api_features.csv').drop(columns='Unnamed: 0')
print(f'Base API: {df_api.shape}')

Base API: (846562, 29)


In [12]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846562 entries, 0 to 846561
Data columns (total 29 columns):
AirBagLocFront              797265 non-null object
BodyClass                   845535 non-null object
BusFloorConfigType          843524 non-null object
BusType                     843524 non-null object
CustomMotorcycleType        846525 non-null object
DisplacementCC              839096 non-null float64
DisplacementCI              839096 non-null float64
DisplacementL               839096 non-null float64
Doors                       739918 non-null float64
EngineCylinders             735744 non-null object
EngineHP                    380764 non-null object
EngineKW                    380764 non-null object
ErrorCode                   846550 non-null object
ErrorText                   846550 non-null object
FuelTypePrimary             789859 non-null object
Make                        846525 non-null object
Manufacturer                846525 non-null object
ManufacturerId    

In [13]:
# Identificar columnas con solo valores "Not Applicable" 
notapp_series = df_api\
                    .isin(['Not Applicable'])\
                    .sum()
cols2drop = list(notapp_series[notapp_series > 1].index)

In [14]:
# Identificar columnas con más de un 15% de datos perdidos
null_series = df_api\
                .isnull()\
                .sum()\
                /df_api.shape[0] 
cols2drop += list(null_series[null_series > .15].index)

In [15]:
# Columnas repetidas en la data original
cols2drop += ['ModelYear', 'Make']

In [16]:
# Borrar aquellas columnas 
df_api = df_api.drop(columns=cols2drop)
print(f'Base API: {df_api.shape}')

Base API: (846562, 16)


In [17]:
df_api=df_api.rename(columns={"AirBagLocFront":"Airag_LocFront",
                        "BodyClass":"d_Body_Class",
                        "DisplacementCC":"d_DisplacementCC",
                        "DisplacementCI":"d_DisplacementCI",
                        "DisplacementL":"d_DisplacementL",
                        "Doors":"d_Doors",
                        "EngineCylinders ":"d_EngineCylinders",
                        "EngineHP":"d_EngineHP",
                        "EngineKW":"d_EngineKW",
                        "ErrorCode":"d_ErrorCode",
                        "ErrorText":"d_ErrorText",
                        "FuelTypePrimary":"d_FuelTypePrimary",
                        "Make":"d_Make",
                        "Manufacturer":"d_Manufacturer",
                        "ManufacturerId":"d_ManufacturerId",
                        "Model":"d_Model",
                        "ModelYear":"d_ModelYear",
                        "PlantCity":"d_PlantCity",
                        "PlantCountry":"d_PlantCountry",
                        "TPMS":"d_TPMS",
                        "VIN":"Vin",
                        "VehicleType":"d_VehicleType"})

## 1.3 Unión de Bases

In [18]:
# Unión de bases
df = pd.merge(left=df_data, 
              right=df_api, 
              how='inner',
              on='Vin')
# Dimensiones de la base
print(f'Dataset: {df.shape}')

Dataset: (846644, 24)


In [19]:
# Información general de la base
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 846644 entries, 0 to 846643
Data columns (total 24 columns):
Price                846644 non-null int64
Year                 846644 non-null int64
Mileage              846644 non-null int64
City                 846644 non-null object
State                846644 non-null object
Vin                  846644 non-null object
Make                 846644 non-null object
Model                846644 non-null object
sample               846644 non-null object
Airag_LocFront       797359 non-null object
d_Body_Class         845629 non-null object
d_DisplacementCC     839190 non-null float64
d_DisplacementCI     839190 non-null float64
d_DisplacementL      839190 non-null float64
d_Doors              740010 non-null float64
EngineCylinders      735838 non-null object
d_ErrorCode          846644 non-null object
d_ErrorText          846644 non-null object
d_FuelTypePrimary    789951 non-null object
d_Manufacturer       846619 non-null object
d_Manufa

In [20]:
# Filtrar 30 modelos
modelos = df['Make'].value_counts()
df = df[df['Make'].isin(modelos[modelos > 30].index)]

In [21]:
clase = df['d_Body_Class'].value_counts()
df = df[df['d_Body_Class'].isin(clase[clase > 1000].index)]

In [22]:
df['Age'] = df['Year'] - 2018

# 2. Análisis exploratorio de datos

## 2.1 Análisis de los datos perdidos

In [23]:
#msgno.matrix(df)

# 3. Preproceso

In [24]:
# Liberar Espacio Memoria
del df_api
del df_data
del df_train
del df_test

In [25]:
# Selección de variables para modelos
select_vars = ['Price', 'Mileage', 'sample', 'd_Body_Class', 'Make', 'Age']
#sample_size = df[select_vars].shape[0]

In [26]:
# Muestra aleatoria
df_sample = df[select_vars]\
                .sample(sample_size)\
                .reset_index(drop=True)\

In [29]:
# Instanciar clase para realizar preproceso
df_prep = PrepML(df_sample)

In [31]:
# Realizamos OneHot Encoder a las columnas categóricas seleccionadas
df_prep.one_hot_encoder(['d_Body_Class', 'Make'], drop_first=True)
df_prep.df

,Price,Mileage,sample,Age,d_Body_Class_Sport_Utility_Vehicle__SUV__Multi_Purpose_Vehicle__MPV_,d_Body_Class_Pickup,d_Body_Class_Wagon,d_Body_Class_Hatchback_Liftback_Notchback,d_Body_Class_Coupe,d_Body_Class_Minivan,...,Make_Genesis,Make_Aston,Make_Oldsmobile,Make_Lamborghini,Make_Rolls_Royce,Make_Plymouth,Make_Isuzu,Make_Alfa,Make_McLaren,Make_Lotus
0,14097,44948,train,-4,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,18399,41491,train,-9,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31995,33070,train,-4,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19995,85952,train,-5,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12988,7512,train,-1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,11415,106608,train,-5,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199996,18533,17143,train,-1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199997,44999,41096,train,-3,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199998,21750,25811,train,-2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Removemos outliers (opcional)
#df_prep.remove_outliers(['Price', 'Mileage'], multiplier=1.5)
#df_prep.df

In [33]:
# Estandarizamos variables continuas seleccionadas
df_prep.standard_scaler(['Mileage', 'Age'])
df_prep.df

,Price,sample,d_Body_Class_Sport_Utility_Vehicle__SUV__Multi_Purpose_Vehicle__MPV_,d_Body_Class_Pickup,d_Body_Class_Wagon,d_Body_Class_Hatchback_Liftback_Notchback,d_Body_Class_Coupe,d_Body_Class_Minivan,d_Body_Class_Convertible_Cabriolet,d_Body_Class_Van,...,Make_Oldsmobile,Make_Lamborghini,Make_Rolls_Royce,Make_Plymouth,Make_Isuzu,Make_Alfa,Make_McLaren,Make_Lotus,Mileage,Age
0,14097,train,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.178517,0.203434
1,18399,train,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.262239,-1.267961
2,31995,train,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.466179,0.203434
3,19995,train,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.814519,-0.090845
4,12988,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.085143,1.086270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,11415,train,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.314766,-0.090845
199996,18533,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.851900,1.086270
199997,44999,train,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.271805,0.497712
199998,21750,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.641978,0.791991


In [34]:
# Separar muestras según
X_train, y_train, X_test, y_test = df_prep.to_train_test_samples('sample', 'Price')

Realizado en 1.0s


# 4. Modelamiento 

In [42]:
# Test
from sklearn.linear_model import LinearRegression
linear_reg = MLModel(LinearRegression())
linear_reg.fit(X_train, y_train)

Realizado en 1.0s


In [44]:
linear_reg.metrics(X_test, y_test)

rmse: 8424.046
mae: 5164.762
r2: 0.597


{'rmse': 8424.046, 'mae': 5164.762, 'r2': 0.597}

## 4.1 Ridge Regression

In [48]:
# Establecemos parámetros a evaluar en el modelo
ridge_grid = {'alpha': [0, .001, 0.0001],
              'solver': ['sag', 'sparse_cg']}
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
ridge_reg = MLModel(model=Ridge(fit_intercept=True))
# Implementación del grid search
ridge_reg.grid_search(X_train,
                      y_train,
                      param_grid=ridge_grid,
                      n_jobs=-2,
                      cv=cv)

Mejores parámetros:
{'alpha': 0.001, 'solver': 'sag'}

Realizado en 103.0s


In [49]:
# Métricas
ridge_reg.metrics(X_test, y_test)

rmse: 8463.538
mae: 5190.979
r2: 0.593


{'rmse': 8463.538, 'mae': 5190.979, 'r2': 0.593}

## 4.2 SGD

In [54]:
# Establecemos parámetros a evaluar en el modelo
sgd_grid = {'loss': ['squared_epsilon_insensitive', 'squared_loss'],
            'alpha': [0, 0.0001, 0.00001]
          }
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
sgd_reg = MLModel(model=SGDRegressor(penalty = 'l1',
                                     early_stopping = False,
                                     random_state=rd_seed))
# Implementación del grid search
sgd_reg.grid_search(X_train,
                    y_train,
                    param_grid=sgd_grid,
                    n_jobs=-2,
                    cv=cv)

Mejores parámetros:
{'alpha': 0, 'loss': 'squared_epsilon_insensitive'}

Realizado en 141.0s


In [55]:
sgd_reg.metrics(X_test, y_test)

rmse: 8481.539
mae: 5196.783
r2: 0.591


{'rmse': 8481.539, 'mae': 5196.783, 'r2': 0.591}

## 4.3 LightGBM

In [ ]:
# Establecemos parámetros a evaluar en el modelo
lgb_grid = {'max_depth': [3, 4, 5, 6], 
            'num_leaves': [30, 50, 60]}
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
lgb_reg = MLModel(model=LGBMRegressor(n_jobs=1,
                                      random_state=rd_seed))
# Implementación del grid search
lgb_reg.grid_search(X_train,
                    y_train,
                    param_grid=lgb_grid,
                    n_jobs=-2,
                    cv=cv)

In [ ]:
lgb_reg.metrics(X_test, y_test)

## 4.4 XGBoost

In [ ]:
# Establecemos parámetros a evaluar en el modelo
xgb_grid = {'max_depth': [4, 5, 6, 7], 
            'n_estimators': [ 60, 70, 80]}
# Instanciamos Clase auxiliar para entrenar, ajustar y evaluar modelos de ML
xgb_reg = MLModel(model=XGBRegressor(objective ='reg:squarederror',
                                     n_jobs=1,
                                     seed=rd_seed))
# Implementación del grid search
xgb_reg.grid_search(X_train,
                    y_train,
                    param_grid=xgb_grid,
                    n_jobs=-2,
                    cv=cv)

In [ ]:
# Métricas
xgb_reg.metrics(X_test, y_test)

In [ ]:
# Experimental: Red Neuronal

In [56]:
# Clase para instanciar modelos de Keras como objetos de scikit-learn
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# Regularizador para neuronas
from tensorflow.keras.regularizers import l2
# Arquitectura secuencial para modelos de redes neuronales
from tensorflow.keras.models import Sequential
# Clase para conexciónes densamente pobladas
from tensorflow.keras.layers import Dense

In [74]:
# Norma de regularización con lambda=0.01
ridge_regularizer = l2(0.0005)


# Instanciamos la Red Neuronal
input_dim = (X_train.shape[1],)
model = Sequential()
model.add(Dense(20, activation='relu',
                kernel_regularizer = ridge_regularizer,
                use_bias=True, 
                input_shape=input_dim,
                name='Primera_capa'))

model.add(Dense(10, activation='relu', 
                kernel_regularizer = ridge_regularizer,
                use_bias=True, 
                name='Segunda_capa'))

model.add(Dense(1, activation='linear', use_bias=True, name='output'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Primera_capa (Dense)         (None, 20)                1300      
_________________________________________________________________
Segunda_capa (Dense)         (None, 10)                210       
_________________________________________________________________
output (Dense)               (None, 1)                 11        
Total params: 1,521
Trainable params: 1,521
Non-trainable params: 0
_________________________________________________________________


In [76]:
model.fit(X_train, y_train, batch_size=512, epochs=50, validation_data=(X_test, y_test))

Train on 150088 samples, validate on 49912 samples
Epoch 1/50
150088/150088 [==============================] - 5s 35us/sample - loss: 96055320.2173 - mean_squared_error: 96055288.0000 - val_loss: 85974869.9074 - val_mean_squared_error: 85974864.0000
Epoch 2/50
150088/150088 [==============================] - 6s 39us/sample - loss: 95554072.8792 - mean_squared_error: 95554104.0000 - val_loss: 85569995.2268 - val_mean_squared_error: 85569992.0000
Epoch 3/50
150088/150088 [==============================] - 6s 39us/sample - loss: 95075316.7486 - mean_squared_error: 95075280.0000 - val_loss: 85214268.5251 - val_mean_squared_error: 85214272.0000
Epoch 4/50
150088/150088 [==============================] - 6s 39us/sample - loss: 94618494.8572 - mean_squared_error: 94618464.0000 - val_loss: 84892128.0282 - val_mean_squared_error: 84892120.0000
Epoch 5/50
150088/150088 [==============================] - 6s 38us/sample - loss: 94182389.9545 - mean_squared_error: 94182360.0000 - val_loss: 84495086